# **Data Preparations**

In [1]:
from google.cloud import bigquery
import pandas as pd

# Initialize a BigQuery client
client_bmi = bigquery.Client(project='lunar-brace-421318')

# Define your query
query_bmi = """
WITH weights AS (
  SELECT
    subject_id,
    hadm_id,
    AVG(CASE
      WHEN valueuom = 'lb' THEN valuenum * 0.453592
      WHEN valueuom = 'kg' THEN valuenum
      ELSE NULL
    END) AS avg_weight_kg
  FROM physionet-data.mimiciii_clinical.chartevents
  WHERE itemid IN (226531, 226512) -- Pounds and kilograms
  GROUP BY subject_id, hadm_id
),
heights AS (
  SELECT
    subject_id,
    hadm_id,
    AVG(CASE
      WHEN valueuom = 'Inch' THEN valuenum * 0.0254
      WHEN valueuom = 'cm' THEN valuenum * 0.01
      ELSE NULL
    END) AS avg_height_m
  FROM physionet-data.mimiciii_clinical.chartevents
  WHERE itemid IN (226707, 226730) -- Centimeters and inches
  GROUP BY subject_id, hadm_id
),
bmi AS (
  SELECT
    w.subject_id,
    w.hadm_id,
    w.avg_weight_kg,
    h.avg_height_m,
    w.avg_weight_kg / POWER(h.avg_height_m, 2) AS BMI
  FROM weights w
  JOIN heights h ON w.subject_id = h.subject_id AND w.hadm_id = h.hadm_id
  WHERE h.avg_height_m != 0 AND w.avg_weight_kg != 0
)

SELECT
  b.subject_id,
  b.hadm_id,
  b.avg_weight_kg AS weight_kg,
  b.avg_height_m AS height_m,
  b.BMI
FROM bmi b
"""

# Run the query and convert the results to a DataFrame
df_bmi = client_bmi.query(query_bmi).to_dataframe()

# Save the DataFrame to a CSV file
df_bmi.to_csv('bmi_results.csv', index=False)

In [2]:
df_bmi.head()

,subject_id,hadm_id,weight_kg,height_m,BMI
0,28067,169880,86.0,1.6001,33.589551
1,50626,151037,77.7,1.6505,28.522656
2,53754,135635,74.8,1.7513,24.388242
3,67556,109843,86.4,1.7513,28.170376
4,46060,123807,62.8,1.5724,25.399992


In [3]:
from google.cloud import bigquery
import pandas as pd

# Initialize a BigQuery client
client_bmi = bigquery.Client(project='lunar-brace-421318')

# Define your query
query_bmi = """
WITH weights AS (
  SELECT
    subject_id,
    hadm_id,
    AVG(CASE
      WHEN valueuom = 'lb' THEN valuenum * 0.453592
      WHEN valueuom = 'kg' THEN valuenum
      ELSE NULL
    END) AS avg_weight_kg
  FROM physionet-data.mimiciii_clinical.chartevents
  WHERE itemid IN (226531, 226512) -- Pounds and kilograms
  GROUP BY subject_id, hadm_id
),
heights AS (
  SELECT
    subject_id,
    hadm_id,
    AVG(CASE
      WHEN valueuom = 'Inch' THEN valuenum * 0.0254
      WHEN valueuom = 'cm' THEN valuenum * 0.01
      ELSE NULL
    END) AS avg_height_m
  FROM physionet-data.mimiciii_clinical.chartevents
  WHERE itemid IN (226707, 226730) -- Centimeters and inches
  GROUP BY subject_id, hadm_id
),
bmi AS (
  SELECT
    w.subject_id,
    w.hadm_id,
    w.avg_weight_kg,
    h.avg_height_m,
    w.avg_weight_kg / POWER(h.avg_height_m, 2) AS BMI
  FROM weights w
  JOIN heights h ON w.subject_id = h.subject_id AND w.hadm_id = h.hadm_id
  WHERE h.avg_height_m != 0 AND w.avg_weight_kg != 0
)

SELECT
  b.subject_id,
  b.hadm_id,
  b.BMI
FROM bmi b
"""
#  b.avg_weight_kg AS weight_kg,b.avg_height_m AS height_m,
# Run the query and convert the results to a DataFrame
df_bmi = client_bmi.query(query_bmi).to_dataframe()

# Save the DataFrame to a CSV file
df_bmi.to_csv('bmi_results.csv', index=False)

In [4]:
from google.cloud import bigquery
import pandas as pd

# Initialize a BigQuery client
client = bigquery.Client(project='lunar-brace-421318')

# Define your query
query = """
WITH hgb AS (
  SELECT subject_id, hadm_id, AVG(valuenum) AS Avg_Hgb
  FROM `physionet-data.mimiciii_clinical.chartevents`
  WHERE itemid = 220228
  GROUP BY subject_id, hadm_id
),
hct AS (
  SELECT subject_id, hadm_id, AVG(valuenum) AS Avg_Hct
  FROM `physionet-data.mimiciii_clinical.chartevents`
  WHERE itemid = 220545
  GROUP BY subject_id, hadm_id
),
wbc AS (
  SELECT subject_id, hadm_id, AVG(valuenum) AS Avg_WBC
  FROM `physionet-data.mimiciii_clinical.chartevents`
  WHERE itemid = 220546
  GROUP BY subject_id, hadm_id
)

SELECT
  hgb.subject_id,
  hgb.hadm_id,
  hgb.Avg_Hgb,
  hct.Avg_Hct,
  wbc.Avg_WBC,
FROM hgb
FULL OUTER JOIN hct ON hgb.subject_id = hct.subject_id AND hgb.hadm_id = hct.hadm_id
FULL OUTER JOIN wbc ON hgb.subject_id = wbc.subject_id AND hgb.hadm_id = wbc.hadm_id
"""

# Run the query and convert the results to a DataFrame
df = client.query(query).to_dataframe()
# Save the DataFrame to a CSV file
df.to_csv('query_results.csv', index=False)

In [5]:
df.head()

,subject_id,hadm_id,Avg_Hgb,Avg_Hct,Avg_WBC
0,6901,101759,12.060,36.145455,10.610
1,57277,152541,8.750,27.750000,10.200
2,20226,135314,8.575,27.475000,20.150
3,27119,147996,13.700,38.100000,9.800
4,26350,113194,10.650,30.420000,7.375


In [6]:
df_bmi.head()

,subject_id,hadm_id,BMI
0,29625,189154,26.103911
1,88445,128410,21.813520
2,54850,122020,31.104790
3,83013,198083,22.014268
4,84624,172515,49.546162


In [7]:
merged_df = pd.merge(df, df_bmi, on=['subject_id','hadm_id'])

In [8]:
merged_df.head()

,subject_id,hadm_id,Avg_Hgb,Avg_Hct,Avg_WBC,BMI
0,6901,101759,12.060000,36.145455,10.6100,25.924101
1,57277,152541,8.750000,27.750000,10.2000,28.277621
2,27119,147996,13.700000,38.100000,9.8000,248.930836
3,92379,136099,11.931250,34.693750,6.8625,31.039790
4,93362,187375,11.366667,33.300000,8.4000,37.430083


In [9]:
merged_df['hadm_id'].is_unique

True

In [10]:
merged_df.duplicated(subset=['subject_id']).sum()

1410

In [11]:
try:
    file_path = 'final_results.csv'  # Adjust this path as necessary
    merged_df.to_csv(file_path, index=False)
    print(f"DataFrame successfully saved to {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

DataFrame successfully saved to final_results.csv


In [12]:
merged_df2 = pd.merge(df, df_bmi, on=['subject_id','hadm_id'],how='inner')

In [13]:
merged_df2[merged_df2.duplicated()]

,subject_id,hadm_id,Avg_Hgb,Avg_Hct,Avg_WBC,BMI


In [14]:
merged_df2.shape

(11703, 6)

In [15]:
merged_df3 = pd.merge(df, df_bmi, on=['subject_id','hadm_id'],how='outer')

In [16]:
merged_df3.shape

(24140, 6)

In [17]:
from google.cloud import bigquery

# Initialize a BigQuery client
client_add = bigquery.Client(project='lunar-brace-421318')

# Define your query
query_add = """
SELECT * FROM `physionet-data.mimiciii_clinical.admissions`
"""
df_add = client_add.query(query_add).to_dataframe()

# Save the DataFrame to a CSV file
df_add.to_csv('add_results.csv', index=False)


In [18]:
df_add.shape

(58976, 19)

In [19]:
df_add.columns = df_add.columns.str.lower()
df_add.head()

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,3757,3115,134067,2139-02-13 03:11:00,2139-02-20 07:33:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,None,None,None,WHITE,2139-02-13 00:02:00,2139-02-13 03:22:00,STAB WOUND,0,1
1,8689,7124,109129,2188-07-11 00:58:00,2188-08-01 12:04:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,None,None,None,WHITE,2188-07-10 14:17:00,2188-07-11 01:52:00,PENILE LACERATION-CELLULITIS,0,1
2,12652,10348,121510,2133-04-16 21:12:00,2133-04-23 15:52:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,None,None,None,UNKNOWN/NOT SPECIFIED,2133-04-16 19:22:00,2133-04-16 22:17:00,STATUS EPILEPTICUS,0,1
3,11501,9396,106469,2109-02-16 23:14:00,2109-02-23 12:01:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,None,None,None,WHITE,2109-02-16 20:58:00,2109-02-16 23:42:00,SUBDURAL HEMATOMA,0,1
4,11419,9333,133732,2167-10-06 18:35:00,2167-10-16 13:13:00,NaT,URGENT,TRANSFER FROM HOSP/EXTRAM,SNF,Private,None,None,None,UNKNOWN/NOT SPECIFIED,NaT,NaT,CORONARY ARTERY DISEASE,0,1


In [20]:
from google.cloud import bigquery

# Initialize a BigQuery client
client_patients = bigquery.Client(project='lunar-brace-421318')

# Define your query
query_patients = """
SELECT * FROM `physionet-data.mimiciii_clinical.patients`
"""
df_patients = client_patients.query(query_patients).to_dataframe()

# Save the DataFrame to a CSV file
# df_patients.to_csv('add_results.csv', index=False)


In [21]:
df_patients.columns = df_patients.columns.str.lower()
df_patients.head()

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,49,56,F,1804-01-02,2104-01-08,2104-01-08,2104-01-08,1
1,17808,18848,F,2042-08-21,2128-01-08,2128-01-08,2128-01-08,1
2,36390,61056,F,2067-04-11,2152-01-08,2152-01-08,2152-01-08,1
3,25371,26889,F,2115-11-04,2164-01-08,2164-01-08,2164-01-08,1
4,17328,18333,F,2094-01-21,2168-01-08,2168-01-08,2168-01-08,1


In [22]:
df_patients.shape

(46520, 8)

In [23]:
from google.cloud import bigquery

# Initialize a BigQuery client
client_diagnoses = bigquery.Client(project='lunar-brace-421318')

# Define your query
query_diagnoses = """
SELECT * FROM `physionet-data.mimiciii_clinical.diagnoses_icd`
"""
df_diagnoses= client_diagnoses.query(query_diagnoses).to_dataframe()


In [24]:
df_diagnoses.columns = df_diagnoses.columns.str.lower()

In [25]:
df_diagnoses.head()

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,3113,256,108811,1,53240
1,3114,256,108811,2,41071
2,3115,256,108811,3,53560
3,3116,256,108811,4,40390
4,3117,256,108811,5,5859


# **Merge create data**


In [26]:
# First, filter to get primary diagnoses where seq_num == 1
primary_diagnoses = df_diagnoses[df_diagnoses['seq_num'] == 1].copy()

# Rename the 'icd9_code' column to 'primary_icd9_code'
primary_diagnoses = primary_diagnoses.rename(columns={"icd9_code": "primary_icd9_code"})

# Calculate the number of unique ICD-9 codes for each subject_id in the original DataFrame
unique_icd9_counts = df_diagnoses.groupby('hadm_id')['icd9_code'].nunique().reset_index()
unique_icd9_counts.rename(columns={'icd9_code': 'num_of_icd9_codes'}, inplace=True)

# Merge this count with the primary_diagnoses DataFrame
primary_diagnoses = primary_diagnoses.merge(unique_icd9_counts, on='hadm_id', how='outer')

# Display the head of the updated DataFrame
primary_diagnoses.head()


,row_id,subject_id,hadm_id,seq_num,primary_icd9_code,num_of_icd9_codes
0,3113,256,108811,1,53240,11
1,3124,256,153771,1,00845,9
2,3133,256,155415,1,4414,6
3,3139,256,188869,1,4413,14
4,6035,512,102509,1,V3101,4


In [27]:
primary_diagnoses.shape

(58976, 6)

In [28]:
df_merged_addp = pd.merge(df_add, df_patients, on='subject_id', how='inner')
df_merged_addp.head(5)

,row_id_x,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,...,diagnosis,hospital_expire_flag,has_chartevents_data,row_id_y,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,3757,3115,134067,2139-02-13 03:11:00,2139-02-20 07:33:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,STAB WOUND,0,1,2950,M,2096-06-07,NaT,NaT,NaT,0
1,8689,7124,109129,2188-07-11 00:58:00,2188-08-01 12:04:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,PENILE LACERATION-CELLULITIS,0,1,6738,M,2135-11-25,2188-08-02,NaT,2188-08-02,1
2,12652,10348,121510,2133-04-16 21:12:00,2133-04-23 15:52:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,STATUS EPILEPTICUS,0,1,9793,F,2049-06-08,2133-06-19,NaT,2133-06-19,1
3,11501,9396,106469,2109-02-16 23:14:00,2109-02-23 12:01:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,SUBDURAL HEMATOMA,0,1,8893,M,1809-02-16,2109-08-07,NaT,2109-08-07,1
4,11419,9333,133732,2167-10-06 18:35:00,2167-10-16 13:13:00,NaT,URGENT,TRANSFER FROM HOSP/EXTRAM,SNF,Private,...,CORONARY ARTERY DISEASE,0,1,8832,F,2107-03-10,NaT,NaT,NaT,0


In [29]:
df_merged_addp.shape

(58976, 26)

In [30]:
df_merged_addpdiag = pd.merge(df_merged_addp, primary_diagnoses, on='hadm_id', how='inner')

df_merged_addpdiag.head(5)

,row_id_x,subject_id_x,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,...,dob,dod,dod_hosp,dod_ssn,expire_flag,row_id,subject_id_y,seq_num,primary_icd9_code,num_of_icd9_codes
0,3757,3115,134067,2139-02-13 03:11:00,2139-02-20 07:33:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,2096-06-07,NaT,NaT,NaT,0,34845,3115,1,8601,4
1,8689,7124,109129,2188-07-11 00:58:00,2188-08-01 12:04:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,2135-11-25,2188-08-02,NaT,2188-08-02,1,79415,7124,1,0389,9
2,12652,10348,121510,2133-04-16 21:12:00,2133-04-23 15:52:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,2049-06-08,2133-06-19,NaT,2133-06-19,1,115974,10348,1,3453,9
3,11501,9396,106469,2109-02-16 23:14:00,2109-02-23 12:01:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,1809-02-16,2109-08-07,NaT,2109-08-07,1,105304,9396,1,85221,7
4,11419,9333,133732,2167-10-06 18:35:00,2167-10-16 13:13:00,NaT,URGENT,TRANSFER FROM HOSP/EXTRAM,SNF,Private,...,2107-03-10,NaT,NaT,NaT,0,104534,9333,1,41071,4


In [31]:
df_merged_addpdiag.shape

(58976, 31)

In [32]:
# Extract year directly from dates as integers
df_merged_addpdiag['dob_year'] = pd.to_datetime(df_merged_addpdiag['dob']).dt.year
df_merged_addpdiag['admittime_year']= pd.to_datetime(df_merged_addpdiag['admittime']).dt.year

# Calculate age by subtracting the year directly
df_merged_addpdiag['age'] = df_merged_addpdiag['admittime_year'] - df_merged_addpdiag['dob_year']

In [33]:
df_merged_addpdiag = df_merged_addpdiag[df_merged_addpdiag['age'] < 300]
# Assuming you've loaded your DataFrame as result
df_merged_addpdiag['admittime'] = pd.to_datetime(df_merged_addpdiag['admittime'])
df_merged_addpdiag['dischtime'] = pd.to_datetime(df_merged_addpdiag['dischtime'])

# Calculate the difference in hours (integer)
df_merged_addpdiag['hospital_stay_hours'] = ((df_merged_addpdiag['dischtime'] - df_merged_addpdiag['admittime']).dt.total_seconds() / 3600).astype(int)
# Check for any negative values which might indicate data issues
# print("Negative hospital stays:", result[result['hospital_stay_hours'] < 0])

# You might want to set negative values to NaN or handle them accordingly
df_merged_addpdiag.loc[df_merged_addpdiag['hospital_stay_hours'] < 0, 'hospital_stay_hours'] = pd.NA
# Display some of the calculated hospital stays to verify
# print(df_merged_addpdiag[['admittime', 'dischtime', 'hospital_stay_hours']].hdf_merged_addpdiag

<ipython-input-33-dab5e9eb4dea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_addpdiag['admittime'] = pd.to_datetime(df_merged_addpdiag['admittime'])
<ipython-input-33-dab5e9eb4dea>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_addpdiag['dischtime'] = pd.to_datetime(df_merged_addpdiag['dischtime'])
<ipython-input-33-dab5e9eb4dea>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [34]:
df_merged_addpdiag.head()

,row_id_x,subject_id_x,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,...,expire_flag,row_id,subject_id_y,seq_num,primary_icd9_code,num_of_icd9_codes,dob_year,admittime_year,age,hospital_stay_hours
0,3757,3115,134067,2139-02-13 03:11:00,2139-02-20 07:33:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,0,34845,3115,1,8601,4,2096,2139,43,172.0
1,8689,7124,109129,2188-07-11 00:58:00,2188-08-01 12:04:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,1,79415,7124,1,0389,9,2135,2188,53,515.0
2,12652,10348,121510,2133-04-16 21:12:00,2133-04-23 15:52:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,1,115974,10348,1,3453,9,2049,2133,84,162.0
4,11419,9333,133732,2167-10-06 18:35:00,2167-10-16 13:13:00,NaT,URGENT,TRANSFER FROM HOSP/EXTRAM,SNF,Private,...,0,104534,9333,1,41071,4,2107,2167,60,234.0
5,25262,20691,119601,2198-02-09 14:58:00,2198-02-20 15:33:00,NaT,URGENT,TRANSFER FROM HOSP/EXTRAM,SNF,Private,...,1,230968,20691,1,41401,6,2131,2198,67,264.0


In [35]:
df_bmi.head()

,subject_id,hadm_id,BMI
0,29625,189154,26.103911
1,88445,128410,21.813520
2,54850,122020,31.104790
3,83013,198083,22.014268
4,84624,172515,49.546162


In [36]:
df.head()

,subject_id,hadm_id,Avg_Hgb,Avg_Hct,Avg_WBC
0,6901,101759,12.060,36.145455,10.610
1,57277,152541,8.750,27.750000,10.200
2,20226,135314,8.575,27.475000,20.150
3,27119,147996,13.700,38.100000,9.800
4,26350,113194,10.650,30.420000,7.375


In [37]:
df_merged_bmicbc= pd.merge(df, df_bmi, on=['subject_id','hadm_id'],how='outer')

In [38]:
df_merged_bmicbc.head()

,subject_id,hadm_id,Avg_Hgb,Avg_Hct,Avg_WBC,BMI
0,6901,101759,12.060,36.145455,10.610,25.924101
1,57277,152541,8.750,27.750000,10.200,28.277621
2,20226,135314,8.575,27.475000,20.150,NaN
3,27119,147996,13.700,38.100000,9.800,248.930836
4,26350,113194,10.650,30.420000,7.375,NaN


In [39]:
result = pd.merge(df_merged_addpdiag, df_merged_bmicbc, on='hadm_id', how='left')

In [40]:
result.head()

,row_id_x,subject_id_x,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,...,num_of_icd9_codes,dob_year,admittime_year,age,hospital_stay_hours,subject_id,Avg_Hgb,Avg_Hct,Avg_WBC,BMI
0,3757,3115,134067,2139-02-13 03:11:00,2139-02-20 07:33:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,4,2096,2139,43,172.0,<NA>,NaN,NaN,NaN,NaN
1,8689,7124,109129,2188-07-11 00:58:00,2188-08-01 12:04:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,9,2135,2188,53,515.0,<NA>,NaN,NaN,NaN,NaN
2,12652,10348,121510,2133-04-16 21:12:00,2133-04-23 15:52:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,9,2049,2133,84,162.0,<NA>,NaN,NaN,NaN,NaN
3,11419,9333,133732,2167-10-06 18:35:00,2167-10-16 13:13:00,NaT,URGENT,TRANSFER FROM HOSP/EXTRAM,SNF,Private,...,4,2107,2167,60,234.0,<NA>,NaN,NaN,NaN,NaN
4,25262,20691,119601,2198-02-09 14:58:00,2198-02-20 15:33:00,NaT,URGENT,TRANSFER FROM HOSP/EXTRAM,SNF,Private,...,6,2131,2198,67,264.0,<NA>,NaN,NaN,NaN,NaN


In [41]:
result.shape

(56360, 40)

In [42]:
print(result.dtypes)

row_id_x                         Int64
subject_id_x                     Int64
hadm_id                          Int64
admittime               datetime64[us]
dischtime               datetime64[us]
deathtime               datetime64[us]
admission_type                  object
admission_location              object
discharge_location              object
insurance                       object
language                        object
religion                        object
marital_status                  object
ethnicity                       object
edregtime               datetime64[us]
edouttime               datetime64[us]
diagnosis                       object
hospital_expire_flag             Int64
has_chartevents_data             Int64
row_id_y                         Int64
gender                          object
dob                     datetime64[us]
dod                     datetime64[us]
dod_hosp                datetime64[us]
dod_ssn                 datetime64[us]
expire_flag              

In [43]:
result['primary_icd9_code']

0         8601
1         0389
2         3453
3        41071
4        41401
         ...  
56355    41401
56356    80604
56357     4271
56358     0389
56359    42731
Name: primary_icd9_code, Length: 56360, dtype: object

In [44]:
result['primary_icd9_code'].isna().sum()

45

In [45]:
result = result.dropna(subset=['primary_icd9_code'])

In [46]:
def classify_icd9(code):
    try:
        # Extract the first three digits as an integer
        code_int = int(code[:3])  # Only first three digits


        if 1 <= code_int <= 139:
            return "001-139"  # infectious and parasitic diseases
        elif 140 <= code_int <= 239:
            return "140-239"  # neoplasms
        elif 240 <= code_int <= 279:
            return "240-279"  # endocrine, nutritional and metabolic diseases, and immunity disorders
        elif 280 <= code_int <= 289:
            return "280-289"  # diseases of the blood and blood-forming organs
        elif 290 <= code_int <= 319:
            return "290-319"  # mental disorders
        elif 320 <= code_int <= 389:
            return "320-389"  # diseases of the nervous system and sense organs
        elif 390 <= code_int <= 459:
            return "390-459"  # diseases of the circulatory system
        elif 460 <= code_int <= 519:
            return "460-519"  # diseases of the respiratory system
        elif 520 <= code_int <= 579:
            return "520-579"  # diseases of the digestive system
        elif 580 <= code_int <= 629:
            return "580-629"  # diseases of the genitourinary system
        elif 630 <= code_int <= 679:
            return "630-679"  # complications of pregnancy, childbirth, and the puerperium
        elif 680 <= code_int <= 709:
            return "680-709"  # diseases of the skin and subcutaneous tissue
        elif 710 <= code_int <= 739:
            return "710-739"  # diseases of the musculoskeletal system and connective tissue
        elif 740 <= code_int <= 759:
            return "740-759"  # congenital anomalies
        elif 760 <= code_int <= 779:
            return "760-779"  # certain conditions originating in the perinatal period
        elif 780 <= code_int <= 799:
            return "780-799"  # symptoms, signs, and ill-defined conditions
        elif 800 <= code_int <= 999:
            return "800-999"  # injury and poisoning
        else:
            return "Unknown"
    except ValueError:
        # Handle V codes
        if code.startswith('V'):
            return "V codes"  # Supplemental classification
        else:
            return "Unknown"


result['primary_icd9_label'] = result['primary_icd9_code'].apply(classify_icd9)

result.head()

,row_id_x,subject_id_x,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,...,dob_year,admittime_year,age,hospital_stay_hours,subject_id,Avg_Hgb,Avg_Hct,Avg_WBC,BMI,primary_icd9_label
0,3757,3115,134067,2139-02-13 03:11:00,2139-02-20 07:33:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,2096,2139,43,172.0,<NA>,NaN,NaN,NaN,NaN,800-999
1,8689,7124,109129,2188-07-11 00:58:00,2188-08-01 12:04:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,2135,2188,53,515.0,<NA>,NaN,NaN,NaN,NaN,001-139
2,12652,10348,121510,2133-04-16 21:12:00,2133-04-23 15:52:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,2049,2133,84,162.0,<NA>,NaN,NaN,NaN,NaN,320-389
3,11419,9333,133732,2167-10-06 18:35:00,2167-10-16 13:13:00,NaT,URGENT,TRANSFER FROM HOSP/EXTRAM,SNF,Private,...,2107,2167,60,234.0,<NA>,NaN,NaN,NaN,NaN,390-459
4,25262,20691,119601,2198-02-09 14:58:00,2198-02-20 15:33:00,NaT,URGENT,TRANSFER FROM HOSP/EXTRAM,SNF,Private,...,2131,2198,67,264.0,<NA>,NaN,NaN,NaN,NaN,390-459


In [47]:
# Drop duplicates if necessary
result = result.drop_duplicates()
result.columns

Index(['row_id_x', 'subject_id_x', 'hadm_id', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data', 'row_id_y', 'gender',
       'dob', 'dod', 'dod_hosp', 'dod_ssn', 'expire_flag', 'row_id',
       'subject_id_y', 'seq_num', 'primary_icd9_code', 'num_of_icd9_codes',
       'dob_year', 'admittime_year', 'age', 'hospital_stay_hours',
       'subject_id', 'Avg_Hgb', 'Avg_Hct', 'Avg_WBC', 'BMI',
       'primary_icd9_label'],
      dtype='object')

In [49]:
columns_to_keep = [ 'gender', 'age','ethnicity', 'admission_type', 'primary_icd9_code', 'primary_icd9_label', 'num_of_icd9_codes', 'hospital_expire_flag', 'hospital_stay_hours', 'BMI','Avg_Hgb','Avg_Hct','Avg_WBC']

# Create a new DataFrame with only the selected columns

final_data = result[columns_to_keep]
final_data = final_data.rename(columns={"Avg_Hgb" : "Hemoglobine","Avg_Hct" : "Hematocrit","Avg_WBC" : "WBC"})
final_data.head(5)

,gender,age,ethnicity,admission_type,primary_icd9_code,primary_icd9_label,num_of_icd9_codes,hospital_expire_flag,hospital_stay_hours,BMI,Hemoglobine,Hematocrit,WBC
0,M,43,WHITE,EMERGENCY,8601,800-999,4,0,172.0,NaN,NaN,NaN,NaN
1,M,53,WHITE,EMERGENCY,0389,001-139,9,0,515.0,NaN,NaN,NaN,NaN
2,F,84,UNKNOWN/NOT SPECIFIED,EMERGENCY,3453,320-389,9,0,162.0,NaN,NaN,NaN,NaN
3,F,60,UNKNOWN/NOT SPECIFIED,URGENT,41071,390-459,4,0,234.0,NaN,NaN,NaN,NaN
4,F,67,UNKNOWN/NOT SPECIFIED,URGENT,41401,390-459,6,0,264.0,NaN,NaN,NaN,NaN


In [50]:
final_data['primary_icd9_label'].unique()

array(['800-999', '001-139', '320-389', '390-459', '240-279', '460-519',
       '780-799', 'V codes', '760-779', '520-579', '140-239', '290-319',
       '740-759', '710-739', '280-289', '580-629', '680-709', '630-679'],
      dtype=object)

In [51]:
# # Converting data types
final_data['hospital_expire_flag'] = final_data['hospital_expire_flag'].astype('boolean')

In [52]:
# Create a dictionary of DataFrames, keyed by the unique labels in 'icd9_label'
tables = {label: final_data[final_data['primary_icd9_label'] == label].copy() for label in final_data['primary_icd9_label'].unique()}

In [53]:
print(final_data['primary_icd9_label'].unique())

['800-999' '001-139' '320-389' '390-459' '240-279' '460-519' '780-799'
 'V codes' '760-779' '520-579' '140-239' '290-319' '740-759' '710-739'
 '280-289' '580-629' '680-709' '630-679']


In [55]:
def classify_icd9_I(code):
    # Extract the first three digits as an integer
    code_int = int(code[:3])  # Only first three digits

    if 1 <= code_int <= 9:
        return "Intestinal infectious diseases"
    elif 10 <= code_int <= 18:
        return "Tuberculosis"
    elif 20 <= code_int <= 27:
        return "Zoonotic bacterial diseases"
    elif 30 <= code_int <= 41:
        return "Other bacterial diseases"
    elif 42 <= code_int <= 44:
        return "Human immunodeficiency virus (HIV) infection"
    elif 45 <= code_int <= 49:
        return "Poliomyelitis and other non-arthropod-borne viral diseases of central nervous system"
    elif 50 <= code_int <= 59:
        return "Viral diseases accompanied by exanthem"
    elif 60 <= code_int <= 66:
        return "Arthropod-borne viral diseases"
    elif 70 <= code_int <= 79:
        return "Other diseases due to viruses and Chlamydiota"
    elif 80 <= code_int <= 88:
        return "Rickettsioses and other arthropod-borne diseases"
    elif 90 <= code_int <= 99:
        return "Syphilis and other venereal diseases"
    elif 100 <= code_int <= 118:
        return "Other spirochetal diseases"
    elif 120 <= code_int <= 129:
        return "Helminthiases"
    elif 130 <= code_int <= 136:
        return "Other infectious and parasitic diseases"
    elif 137 <= code_int <= 139:
        return "Late effects of infectious and parasitic diseases"
    else:
        return "Unknown"


tables["001-139"]['primary_icd9_label'] = tables["001-139"]['primary_icd9_code'].apply(classify_icd9_I)

tables["001-139"].head()

,gender,age,ethnicity,admission_type,primary_icd9_code,primary_icd9_label,num_of_icd9_codes,hospital_expire_flag,hospital_stay_hours,BMI,Hemoglobine,Hematocrit,WBC
1,M,53,WHITE,EMERGENCY,0389,Other bacterial diseases,9,False,515.0,NaN,NaN,NaN,NaN
54,M,41,UNKNOWN/NOT SPECIFIED,URGENT,0389,Other bacterial diseases,9,False,73.0,NaN,NaN,NaN,NaN
91,F,55,BLACK/AFRICAN AMERICAN,EMERGENCY,0389,Other bacterial diseases,9,True,40.0,NaN,NaN,NaN,NaN
115,M,79,UNKNOWN/NOT SPECIFIED,EMERGENCY,0383,Other bacterial diseases,9,True,99.0,NaN,NaN,NaN,NaN
157,M,77,ASIAN,EMERGENCY,0389,Other bacterial diseases,9,False,345.0,NaN,NaN,NaN,NaN


In [56]:
def replace_nan_with_mean_or_preset(df):
    for column in df.columns:
        if df[column].isna().all():
            df = df.drop(columns=column)
        else:
            if pd.api.types.is_numeric_dtype(df[column]):
                mean_value = df[column].mean()
                df[column].fillna(mean_value, inplace=True)
            else:
                mode_value = df[column].mode()[0]
                df[column].fillna(mode_value, inplace=True)
    return df

In [57]:
df_chapter_1 = pd.DataFrame(tables["001-139"])
df_chapter_1 = replace_nan_with_mean_or_preset(df_chapter_1)
# df_chapter_1.to_csv('chapter_1_nanx.csv', index=False)

In [58]:
def classify_icd9_II(code):
    # Extract the first three digits as an integer
    code_int = int(code[:3])  # Only first three digits

    if 140 <= code_int <= 149:
        return "Malignant neoplasm of lip, oral cavity, and pharynx"
    elif 150 <= code_int <= 159:
        return "Malignant neoplasm of digestive organs and peritoneum"
    elif 160 <= code_int <= 165:
        return "Malignant neoplasm of respiratory and intrathoracic organs"
    elif 170 <= code_int <= 175:
        return "Malignant neoplasm of bone, connective tissue, skin, and breast"
    elif 176 <= code_int <= 176:
        return "Kaposi's sarcoma"
    elif 179 <= code_int <= 189:
        return "Malignant neoplasm of genitourinary organs"
    elif 190 <= code_int <= 199:
        return "Malignant neoplasm of other and unspecified sites"
    elif 200 <= code_int <= 208:
        return "Malignant neoplasm of lymphatic and hematopoietic tissue"
    elif 209 <= code_int <= 209:
        return "Neuroendocrine tumors"
    elif 210 <= code_int <= 229:
        return "Benign neoplasms"
    elif 230 <= code_int <= 234:
        return "Carcinoma in situ "
    elif 235 <= code_int <= 238:
        return "Neoplasms of uncertain behavior"
    elif 239 <= code_int <= 239:
        return "Neoplasms of unspecified nature"
    else:
        return "Unknown"


tables["140-239"]['primary_icd9_label'] = tables["140-239"]['primary_icd9_code'].apply(classify_icd9_II)

tables["140-239"].head()

,gender,age,ethnicity,admission_type,primary_icd9_code,primary_icd9_label,num_of_icd9_codes,hospital_expire_flag,hospital_stay_hours,BMI,Hemoglobine,Hematocrit,WBC
87,F,77,WHITE,EMERGENCY,1974,Malignant neoplasm of other and unspecified sites,8,True,431.0,NaN,NaN,NaN,NaN
97,F,66,WHITE,EMERGENCY,1536,Malignant neoplasm of digestive organs and per...,9,True,229.0,NaN,NaN,NaN,NaN
105,M,63,UNKNOWN/NOT SPECIFIED,URGENT,1970,Malignant neoplasm of other and unspecified sites,5,True,183.0,NaN,NaN,NaN,NaN
142,M,80,HISPANIC OR LATINO,EMERGENCY,1983,Malignant neoplasm of other and unspecified sites,13,False,207.0,NaN,12.0,34.5,12.4
173,F,85,UNKNOWN/NOT SPECIFIED,EMERGENCY,1830,Malignant neoplasm of genitourinary organs,9,False,239.0,NaN,NaN,NaN,NaN


In [59]:
df_chapter_2 = pd.DataFrame(tables["140-239"])
df_chapter_2 = replace_nan_with_mean_or_preset(df_chapter_2)
# df_chapter_2.to_csv('chapter_2_nanx.csv', index=False)

In [60]:
def classify_icd9_III(code):
    # Extract the first three digits as an integer
    code_int = int(code[:3])  # Only first three digits

    if 240 <= code_int <= 246:
        return "Disorders of thyroid gland"
    elif 249 <= code_int <= 259:
        return "Diseases of other endocrine glands"
    elif 260 <= code_int <= 269:
        return "Nutritional deficiencies"
    elif 270 <= code_int <= 279:
        return "Other metabolic and immunity disorders"
    else:
        return "Unknown"


tables["240-279"]['primary_icd9_label'] = tables["240-279"]['primary_icd9_code'].apply(classify_icd9_III)

tables["240-279"].head()

,gender,age,ethnicity,admission_type,primary_icd9_code,primary_icd9_label,num_of_icd9_codes,hospital_expire_flag,hospital_stay_hours,BMI,Hemoglobine,Hematocrit,WBC
15,M,40,WHITE,EMERGENCY,25011,Diseases of other endocrine glands,9,False,289.0,NaN,NaN,NaN,NaN
275,M,62,WHITE,EMERGENCY,2555,Diseases of other endocrine glands,39,True,808.0,NaN,NaN,NaN,NaN
613,M,64,WHITE,EMERGENCY,27541,Other metabolic and immunity disorders,23,False,129.0,NaN,NaN,NaN,NaN
674,M,54,WHITE,EMERGENCY,27651,Other metabolic and immunity disorders,9,False,16.0,NaN,NaN,NaN,NaN
679,M,87,WHITE,EMERGENCY,2449,Disorders of thyroid gland,15,False,144.0,NaN,NaN,NaN,NaN


In [61]:
df_chapter_3 = pd.DataFrame(tables["240-279"])
df_chapter_3 = replace_nan_with_mean_or_preset(df_chapter_3)
# df_chapter_3.to_csv('chapter_3_nanx.csv', index=False)

In [62]:
def classify_icd9_IV(code):
    # Extract the first three digits as an integer
    code_int = int(code[:3])  # Only first three digits

    if 280 <= code_int <= 285:
        return "Anemia"
    elif 286 <= code_int <= 287:
        return "Coagulation/hemorrhagic"
    elif 288 <= code_int <= 289:
        return "Other disease of the blood"
    else:
        return "Unknown"


tables["280-289"]['primary_icd9_label'] = tables["280-289"]['primary_icd9_code'].apply(classify_icd9_IV)

tables["280-289"].head()

,gender,age,ethnicity,admission_type,primary_icd9_code,primary_icd9_label,num_of_icd9_codes,hospital_expire_flag,hospital_stay_hours,BMI,Hemoglobine,Hematocrit,WBC
1290,M,51,WHITE,EMERGENCY,2884,Other disease of the blood,26,False,517.0,41.582258,8.452632,23.619048,1.5
10890,F,88,WHITE,EMERGENCY,28800,Other disease of the blood,10,False,236.0,NaN,8.700000,25.200000,0.9
11051,M,29,WHITE,EMERGENCY,28959,Other disease of the blood,2,False,106.0,NaN,NaN,NaN,NaN
11202,F,85,WHITE,EMERGENCY,2859,Anemia,16,False,42.0,NaN,8.200000,25.000000,6.4
11351,M,76,WHITE,EMERGENCY,2800,Anemia,16,False,87.0,NaN,7.700000,25.300000,7.9


In [63]:
df_chapter_4 = pd.DataFrame(tables["280-289"])
df_chapter_4 = replace_nan_with_mean_or_preset(df_chapter_4)
# df_chapter_4.to_csv('chapter_4_nanx.csv', index=False)

In [64]:
def classify_icd9_V(code):
    # Extract the first three digits as an integer
    code_int = int(code[:3])  # Only first three digits

    if 290 <= code_int <= 299:
        return "Psychosis"
    elif 300 <= code_int <= 316:
        return "Neurotic disorders, personality disorders, and other nonpsychotic mental disorders"
    elif 317 <= code_int <= 319:
        return "Mental retardation"
    else:
        return "Unknown"


tables["290-319"]['primary_icd9_label'] = tables["290-319"]['primary_icd9_code'].apply(classify_icd9_V)

tables["290-319"].head()

,gender,age,ethnicity,admission_type,primary_icd9_code,primary_icd9_label,num_of_icd9_codes,hospital_expire_flag,hospital_stay_hours,BMI,Hemoglobine,Hematocrit,WBC
123,M,32,UNKNOWN/NOT SPECIFIED,EMERGENCY,30011,"Neurotic disorders, personality disorders, and...",7,False,223.0,NaN,NaN,NaN,NaN
167,F,81,UNKNOWN/NOT SPECIFIED,EMERGENCY,30501,"Neurotic disorders, personality disorders, and...",9,False,284.0,NaN,NaN,NaN,NaN
199,M,42,WHITE,EMERGENCY,29181,Psychosis,4,False,231.0,NaN,NaN,NaN,NaN
287,M,53,WHITE,EMERGENCY,2910,Psychosis,9,False,630.0,NaN,NaN,NaN,NaN
289,M,29,HISPANIC OR LATINO,EMERGENCY,2910,Psychosis,11,False,138.0,NaN,NaN,NaN,NaN


In [ ]:
df_chapter_5 = pd.DataFrame(tables["290-319"])
df_chapter_5 = replace_nan_with_mean_or_preset(df_chapter_5)
# df_chapter_5.to_csv('chapter_5_nanx.csv', index=False)

In [66]:
def classify_icd9_VI(code):
    # Extract the first three digits as an integer
    code_int = int(code[:3])  # Only first three digits

    if 320 <= code_int <= 326:
        return "Inflammatory diseases of the central nervous system"
    elif 330 <= code_int <= 337:
        return "Hereditary and Degenerative diseases of the central nervous system"
    elif 340 <= code_int <= 349:
        return "Other disorders of the central nervous system"
    elif 350 <= code_int <= 359:
        return "Disorders of the peripheral nervous system"
    elif 360 <= code_int <= 379:
        return "Disorders of the eye and adnexa"
    elif 380 <= code_int <= 389:
        return "Diseases of the ear and mastoid process"
    else:
        return "Unknown"


tables["320-389"]['primary_icd9_label'] = tables["320-389"]['primary_icd9_code'].apply(classify_icd9_VI)

tables["320-389"].head()

,gender,age,ethnicity,admission_type,primary_icd9_code,primary_icd9_label,num_of_icd9_codes,hospital_expire_flag,hospital_stay_hours,BMI,Hemoglobine,Hematocrit,WBC
2,F,84,UNKNOWN/NOT SPECIFIED,EMERGENCY,3453,Other disorders of the central nervous system,9,False,162.0,NaN,NaN,NaN,NaN
75,F,42,UNKNOWN/NOT SPECIFIED,EMERGENCY,34591,Other disorders of the central nervous system,9,True,463.0,NaN,NaN,NaN,NaN
113,M,49,UNKNOWN/NOT SPECIFIED,EMERGENCY,32089,Inflammatory diseases of the central nervous s...,9,True,194.0,NaN,NaN,NaN,NaN
307,M,72,UNKNOWN/NOT SPECIFIED,EMERGENCY,34590,Other disorders of the central nervous system,13,False,78.0,NaN,13.866667,42.666667,12.900000
538,F,54,UNKNOWN/NOT SPECIFIED,EMERGENCY,3481,Other disorders of the central nervous system,15,True,149.0,36.381026,10.775000,31.566667,10.741667


In [67]:
df_chapter_6 = pd.DataFrame(tables["320-389"])
df_chapter_6 = replace_nan_with_mean_or_preset(df_chapter_6)
# df_chapter_6.to_csv('chapter_6_nanx.csv', index=False)

In [68]:
def classify_icd9_VII(code):

        # Extract the first three digits as an integer
        code_int = int(code[:3])  # Only first three digits

        if 390<= code_int <= 392:
            return "Acute rheumatic fever"
        elif 393 <= code_int <= 398:
            return "Chronic rheumatic heart disease"
        elif 401 <= code_int <= 405:
            return "Hypertensive disease"
        elif 410 <= code_int <= 414:
            return "Ischemic heart disease"
        elif 415 <= code_int <= 417:
            return "Diseases of pulmonary circulation"
        elif 420 <= code_int <= 429:
            return "Other forms of heart disease"
        elif 430 <= code_int <= 438:
            return "Cerebrovascular disease"
        elif 440 <= code_int <= 449:
            return "Diseases of arteries, arterioles, and capillaries"
        elif 451 <= code_int <= 459:
            return "Diseases of veins and lymphatics, and other diseases of circulatory system "
        else:
            return "Unknown"

tables["390-459"]['primary_icd9_label'] = tables["390-459"]['primary_icd9_code'].apply(classify_icd9_VII)
tables["390-459"].head()

,gender,age,ethnicity,admission_type,primary_icd9_code,primary_icd9_label,num_of_icd9_codes,hospital_expire_flag,hospital_stay_hours,BMI,Hemoglobine,Hematocrit,WBC
3,F,60,UNKNOWN/NOT SPECIFIED,URGENT,41071,Ischemic heart disease,4,False,234.0,NaN,NaN,NaN,NaN
4,F,67,UNKNOWN/NOT SPECIFIED,URGENT,41401,Ischemic heart disease,6,False,264.0,NaN,NaN,NaN,NaN
6,F,51,BLACK/AFRICAN AMERICAN,EMERGENCY,4280,Other forms of heart disease,9,False,212.0,NaN,NaN,NaN,NaN
10,F,71,UNKNOWN/NOT SPECIFIED,EMERGENCY,41401,Ischemic heart disease,8,False,165.0,NaN,NaN,NaN,NaN
18,M,39,WHITE,EMERGENCY,41041,Ischemic heart disease,6,False,80.0,NaN,NaN,NaN,NaN


In [69]:
df_chapter_7 = pd.DataFrame(tables["390-459"])
df_chapter_7 = replace_nan_with_mean_or_preset(df_chapter_7)
# df_chapter_7.to_csv('chapter_7_nanx.csv', index=False)

In [70]:
def classify_icd9_VIII(code):

        # Extract the first three digits as an integer
        code_int = int(code[:3])  # Only first three digits

        if 460<= code_int <= 466:
            return "Acute respiratory infections"
        elif 470 <= code_int <= 478:
            return "Other diseases of the upper respiratory tract"
        elif 480 <= code_int <= 488:
            return "Pneumonia and influenza"
        elif 490 <= code_int <= 496:
            return "Chronic obstructive pulmonary disease and allied conditions"
        elif 500 <= code_int <= 508:
            return "Pneumoconioses and other lung diseases due to external agents"
        elif 510 <= code_int <= 519:
            return "Other diseases of respiratory system"
        else:
          return "Unknown"

tables["460-519"]['primary_icd9_label'] = tables["460-519"]['primary_icd9_code'].apply(classify_icd9_VIII)
tables["460-519"].head()

,gender,age,ethnicity,admission_type,primary_icd9_code,primary_icd9_label,num_of_icd9_codes,hospital_expire_flag,hospital_stay_hours,BMI,Hemoglobine,Hematocrit,WBC
17,F,23,WHITE,EMERGENCY,51881,Other diseases of respiratory system,3,False,10.0,NaN,NaN,NaN,NaN
30,F,78,HISPANIC OR LATINO,EMERGENCY,4784,Other diseases of the upper respiratory tract,5,False,15.0,NaN,NaN,NaN,NaN
81,M,68,WHITE,EMERGENCY,5070,Pneumoconioses and other lung diseases due to ...,9,True,569.0,NaN,NaN,NaN,NaN
117,F,73,UNKNOWN/NOT SPECIFIED,URGENT,48239,Pneumonia and influenza,9,True,235.0,NaN,NaN,NaN,NaN
118,F,75,UNKNOWN/NOT SPECIFIED,EMERGENCY,49121,Chronic obstructive pulmonary disease and alli...,6,True,385.0,NaN,NaN,NaN,NaN


In [71]:
df_chapter_8 = pd.DataFrame(tables["460-519"])
df_chapter_8 = replace_nan_with_mean_or_preset(df_chapter_8)
# df_chapter_8.to_csv('chapter_8_nanx.csv', index=False)

In [72]:
def classify_icd9_IX(code):

        # Extract the first three digits as an integer
        code_int = int(code[:3])  # Only first three digits

        if 520<= code_int <= 529:
            return "Diseases of oral cavity, salivary glands, and jaws"
        elif 530 <= code_int <= 539:
            return "Diseases of esophagus, stomach, and duodenum"
        elif 540 <= code_int <= 543:
            return "Appendicitis "
        elif 550 <= code_int <= 553:
            return "Hernia of abdominal cavity"
        elif 560 <= code_int <= 569:
            return "Other diseases of intestines and peritoneum"
        elif 570 <= code_int <= 579:
            return "Other diseases of digestive system"
        else:
          return "Unknown"

tables["520-579"]['primary_icd9_label'] = tables["520-579"]['primary_icd9_code'].apply(classify_icd9_IX)
tables["520-579"].head()

,gender,age,ethnicity,admission_type,primary_icd9_code,primary_icd9_label,num_of_icd9_codes,hospital_expire_flag,hospital_stay_hours,BMI,Hemoglobine,Hematocrit,WBC
63,M,62,UNKNOWN/NOT SPECIFIED,EMERGENCY,56212,Other diseases of intestines and peritoneum,4,False,42.0,NaN,NaN,NaN,NaN
64,M,61,UNKNOWN/NOT SPECIFIED,URGENT,5789,Other diseases of digestive system,8,False,73.0,NaN,NaN,NaN,NaN
80,F,79,UNKNOWN/NOT SPECIFIED,EMERGENCY,5715,Other diseases of digestive system,4,True,125.0,NaN,NaN,NaN,NaN
106,M,88,UNKNOWN/NOT SPECIFIED,URGENT,5571,Unknown,9,True,359.0,NaN,NaN,NaN,NaN
108,M,50,UNKNOWN/NOT SPECIFIED,EMERGENCY,5715,Other diseases of digestive system,9,True,612.0,NaN,NaN,NaN,NaN


In [73]:
df_chapter_9 = pd.DataFrame(tables["520-579"])
df_chapter_9 = replace_nan_with_mean_or_preset(df_chapter_9)
# df_chapter_9.to_csv('chapter_9_nanx.csv', index=False)

In [74]:
def classify_icd9_X(code):

        # Extract the first three digits as an integer
        code_int = int(code[:3])  # Only first three digits

        if 580<= code_int <= 589:
            return "Nephritis, nephrotic syndrome, and nephrosis"
        elif 590 <= code_int <= 599:
            return "Other diseases of urinary system"
        elif 600 <= code_int <= 608:
            return "Diseases of male genital organs "
        elif 610 <= code_int <= 612:
            return "Disorders Of Breast "
        elif 614 <= code_int <= 616:
            return "Inflammatory Disease Of Female Pelvic Organs"
        elif 617 <= code_int <= 629:
            return "Other disorders of female genital tract"
        else:
          return "Unknown"

tables["580-629"]['primary_icd9_label'] = tables["580-629"]['primary_icd9_code'].apply(classify_icd9_X)
tables["580-629"].head()

,gender,age,ethnicity,admission_type,primary_icd9_code,primary_icd9_label,num_of_icd9_codes,hospital_expire_flag,hospital_stay_hours,BMI,Hemoglobine,Hematocrit,WBC
1308,F,62,WHITE,EMERGENCY,5849,"Nephritis, nephrotic syndrome, and nephrosis",12,False,76.0,NaN,9.9,30.7,5.2
2759,M,45,UNKNOWN/NOT SPECIFIED,EMERGENCY,5849,"Nephritis, nephrotic syndrome, and nephrosis",5,False,139.0,NaN,NaN,NaN,NaN
3663,M,74,UNKNOWN/NOT SPECIFIED,EMERGENCY,5849,"Nephritis, nephrotic syndrome, and nephrosis",9,False,111.0,NaN,NaN,NaN,NaN
3680,M,84,BLACK/AFRICAN AMERICAN,EMERGENCY,5849,"Nephritis, nephrotic syndrome, and nephrosis",9,False,250.0,NaN,NaN,NaN,NaN
3686,F,82,WHITE,EMERGENCY,5849,"Nephritis, nephrotic syndrome, and nephrosis",9,False,361.0,NaN,NaN,NaN,NaN


In [75]:
df_chapter_10 = pd.DataFrame(tables["580-629"])
df_chapter_10 = replace_nan_with_mean_or_preset(df_chapter_10)
# df_chapter_10.to_csv('chapter_10_nanx.csv', index=False)

In [76]:
def classify_icd9_XI(code):

        # Extract the first three digits as an integer
        code_int = int(code[:3])  # Only first three digits

        if 630<= code_int <= 633:
            return "Ectopic and molar pregnancy"
        elif 634 <= code_int <= 639:
            return "Other pregnancy with abortive outcome"
        elif 640 <= code_int <= 649:
            return "Complications mainly related to pregnancy"
        elif 650 <= code_int <= 659:
            return "Normal delivery, and other indications for care in pregnancy, labor, and delivery"
        elif 660 <= code_int <= 669:
            return "Complications occurring mainly in the course of labor and delivery"
        elif 670 <= code_int <= 677:
            return "Complications of the puerperium"
        elif 678 <= code_int <= 679:
            return "Other maternal and fetal complications"
        else:
          return "Unknown"

tables["630-679"]['primary_icd9_label'] = tables["630-679"]['primary_icd9_code'].apply(classify_icd9_XI)
tables["630-679"].head()

,gender,age,ethnicity,admission_type,primary_icd9_code,primary_icd9_label,num_of_icd9_codes,hospital_expire_flag,hospital_stay_hours,BMI,Hemoglobine,Hematocrit,WBC
4127,F,38,UNKNOWN/NOT SPECIFIED,EMERGENCY,63522,Other pregnancy with abortive outcome,11,False,135.0,NaN,NaN,NaN,NaN
10033,F,33,UNKNOWN/NOT SPECIFIED,URGENT,66612,Complications occurring mainly in the course o...,2,False,110.0,NaN,NaN,NaN,NaN
10254,F,29,OTHER,EMERGENCY,64683,Complications mainly related to pregnancy,5,False,324.0,NaN,NaN,NaN,NaN
10403,F,22,BLACK/AFRICAN AMERICAN,EMERGENCY,64893,Complications mainly related to pregnancy,12,False,144.0,31.952219,9.475,26.75,20.775
11122,F,45,WHITE,URGENT,64251,Complications mainly related to pregnancy,11,False,263.0,NaN,NaN,NaN,NaN


In [77]:
df_chapter_11 = pd.DataFrame(tables["630-679"])
df_chapter_11 = replace_nan_with_mean_or_preset(df_chapter_11)
# df_chapter_11.to_csv('chapter_11_nanx.csv', index=False)

In [78]:
def classify_icd9_XII(code):

        # Extract the first three digits as an integer
        code_int = int(code[:3])  # Only first three digits

        if 680<= code_int <= 686:
            return "Infections of skin and subcutaneous tissue"
        elif 690 <= code_int <= 698:
            return "Other inflammatory conditions of skin and subcutaneous tissue"
        elif 700 <= code_int <= 709:
            return "Other diseases of skin and subcutaneous tissue"

        else:
          return "Unknown"

tables["680-709"]['primary_icd9_label'] = tables["680-709"]['primary_icd9_code'].apply(classify_icd9_XII)
tables["680-709"].head()

,gender,age,ethnicity,admission_type,primary_icd9_code,primary_icd9_label,num_of_icd9_codes,hospital_expire_flag,hospital_stay_hours,BMI,Hemoglobine,Hematocrit,WBC
3218,F,53,UNABLE TO OBTAIN,ELECTIVE,7092,Other diseases of skin and subcutaneous tissue,18,False,267.0,20.910503,8.85,26.080000,11.133333
4154,F,51,UNKNOWN/NOT SPECIFIED,EMERGENCY,7018,Other diseases of skin and subcutaneous tissue,2,False,12.0,NaN,NaN,NaN,NaN
6666,F,81,UNKNOWN/NOT SPECIFIED,ELECTIVE,68110,Infections of skin and subcutaneous tissue,7,False,71.0,NaN,NaN,NaN,NaN
7252,F,85,UNKNOWN/NOT SPECIFIED,EMERGENCY,70709,Other diseases of skin and subcutaneous tissue,15,False,288.0,NaN,12.00,36.066667,8.833333
7577,F,69,UNKNOWN/NOT SPECIFIED,ELECTIVE,7092,Other diseases of skin and subcutaneous tissue,13,False,156.0,45.287585,11.10,34.066667,11.000000


In [79]:
df_chapter_12 = pd.DataFrame(tables["680-709"])
df_chapter_12 = replace_nan_with_mean_or_preset(df_chapter_12)
# df_chapter_12.to_csv('chapter_12_nanx.csv', index=False)

In [80]:
def classify_icd9_XIII(code):

        # Extract the first three digits as an integer
        code_int = int(code[:3])  # Only first three digits

        if 710<= code_int <= 719:
            return "Arthropathies and related disorders"
        elif 720 <= code_int <= 724:
            return "Dorsopathies"
        elif 725 <= code_int <= 729:
            return "Rheumatism, excluding the back"
        elif 730 <= code_int <= 739:
            return "Osteopathies, chondropathies, and acquired musculoskeletal deformities"
        else:
          return "Unknown"

tables["710-739"]['primary_icd9_label'] = tables["710-739"]['primary_icd9_code'].apply(classify_icd9_XIII)
tables["710-739"].head()

,gender,age,ethnicity,admission_type,primary_icd9_code,primary_icd9_label,num_of_icd9_codes,hospital_expire_flag,hospital_stay_hours,BMI,Hemoglobine,Hematocrit,WBC
645,M,48,WHITE,EMERGENCY,72271,Dorsopathies,5,False,231.0,NaN,NaN,NaN,NaN
1271,F,74,UNABLE TO OBTAIN,ELECTIVE,7213,Dorsopathies,15,False,194.0,NaN,8.100,24.785714,6.250
1273,F,72,UNABLE TO OBTAIN,ELECTIVE,7213,Dorsopathies,14,False,269.0,NaN,11.275,32.737500,14.625
1274,M,81,UNABLE TO OBTAIN,ELECTIVE,7213,Dorsopathies,20,False,173.0,NaN,9.200,25.400000,6.900
2755,F,55,WHITE,ELECTIVE,73730,"Osteopathies, chondropathies, and acquired mus...",11,False,198.0,23.398585,9.600,27.500000,5.875


In [81]:
df_chapter_13 = pd.DataFrame(tables["710-739"])
df_chapter_13 = replace_nan_with_mean_or_preset(df_chapter_13)
# df_chapter_13.to_csv('chapter_13_nanx.csv', index=False)

In [82]:
def classify_icd9_XVI(code):

        # Extract the first three digits as an integer
        code_int = int(code[:3])  # Only first three digits

        if 740<= code_int <= 742:
            return "Nervous system"
        elif 743 <= code_int <= 744:
            return "Eye, ear, face and neck"
        elif 745 <= code_int <= 747:
            return "Circulatory system"
        elif 748 <= code_int <= 748:
            return "Respiratory system"
        elif 749 <= code_int <= 751:
            return "Digestive system"
        elif 752 <= code_int <= 752:
            return "Genital organs"
        elif 753 <= code_int <= 753:
            return "Urinary system"
        elif 754 <= code_int <= 756:
            return "Musculoskeletal system"
        elif 757 <= code_int <= 757:
            return "Integument"
        elif 758 <= code_int <= 758:
            return "Chromosomal anomalies"
        elif 759 <= code_int <= 759:
            return "Other unspecified congenital anomalies"
        else:
          return "Unknown"

tables["740-759"]['primary_icd9_label'] = tables["740-759"]['primary_icd9_code'].apply(classify_icd9_XVI)
tables["740-759"].head()

,gender,age,ethnicity,admission_type,primary_icd9_code,primary_icd9_label,num_of_icd9_codes,hospital_expire_flag,hospital_stay_hours,BMI,Hemoglobine,Hematocrit,WBC
286,M,48,WHITE,EMERGENCY,7424,Nervous system,7,False,538.0,NaN,NaN,NaN,NaN
1036,M,0,WHITE,EMERGENCY,7464,Circulatory system,4,False,71.0,NaN,NaN,NaN,NaN
1809,M,0,WHITE,NEWBORN,74710,Circulatory system,3,False,69.0,NaN,NaN,NaN,NaN
1831,F,0,WHITE,EMERGENCY,7512,Digestive system,3,False,57.0,NaN,NaN,NaN,NaN
2768,F,0,WHITE,NEWBORN,7423,Nervous system,6,False,789.0,NaN,NaN,NaN,NaN


In [83]:
df_chapter_14 = pd.DataFrame(tables["740-759"])
df_chapter_14 = replace_nan_with_mean_or_preset(df_chapter_14)
# df_chapter_14.to_csv('chapter_14_nanx.csv', index=False)

In [90]:
def classify_icd9_XV(code):

        # Extract the first three digits as an integer
        code_int = int(code[:3])  # Only first three digits

        if 760<= code_int <= 763:
            return "Maternal causes of perinatal morbidity and mortality "
        elif 764 <= code_int <= 779:
            return "Other conditions originating in the perinatal period"
        else:
          return "Unknown"

tables["760-779"]['primary_icd9_label'] = tables["760-779"]['primary_icd9_code'].apply(classify_icd9_XV)
tables["760-779"].head()

,gender,age,ethnicity,admission_type,primary_icd9_code,primary_icd9_label,num_of_icd9_codes,hospital_expire_flag,hospital_stay_hours,BMI,Hemoglobine,Hematocrit,WBC
58,F,0,UNKNOWN/NOT SPECIFIED,URGENT,76518,Other conditions originating in the perinatal ...,4,False,149.0,NaN,NaN,NaN,NaN
147,F,0,UNKNOWN/NOT SPECIFIED,URGENT,76504,Other conditions originating in the perinatal ...,8,False,897.0,NaN,NaN,NaN,NaN
231,F,0,ASIAN,EMERGENCY,7746,Other conditions originating in the perinatal ...,1,False,40.0,NaN,NaN,NaN,NaN
353,F,0,WHITE,EMERGENCY,7731,Other conditions originating in the perinatal ...,1,False,45.0,NaN,NaN,NaN,NaN
358,M,0,WHITE,EMERGENCY,7746,Other conditions originating in the perinatal ...,2,False,25.0,NaN,NaN,NaN,NaN


In [91]:
df_chapter_15 = pd.DataFrame(tables["760-779"])
df_chapter_15 = replace_nan_with_mean_or_preset(df_chapter_15)
# df_chapter_15.to_csv('chapter_15_nanx.csv', index=False)

In [86]:
def classify_icd9_XVI(code):

        # Extract the first three digits as an integer
        code_int = int(code[:3])  # Only first three digits

        if 780<= code_int <= 789:
            return "Symptoms"
        elif 790 <= code_int <= 796:
            return "Nonspecific abnormal findings"
        elif 797 <= code_int <= 799:
            return "Ill-defined and unknown causes of morbidity and mortality"
        else:
          return "Unknown"

tables["780-799"]['primary_icd9_label'] = tables["780-799"]['primary_icd9_code'].apply(classify_icd9_XVI)
tables["780-799"].head()

,gender,age,ethnicity,admission_type,primary_icd9_code,primary_icd9_label,num_of_icd9_codes,hospital_expire_flag,hospital_stay_hours,BMI,Hemoglobine,Hematocrit,WBC
24,M,19,WHITE,EMERGENCY,78009,Symptoms,4,False,14.0,NaN,NaN,NaN,NaN
73,F,55,WHITE,EMERGENCY,78039,Symptoms,9,True,119.0,NaN,NaN,NaN,NaN
197,M,64,UNKNOWN/NOT SPECIFIED,EMERGENCY,7809,Symptoms,9,False,89.0,NaN,NaN,NaN,NaN
1039,M,0,WHITE,EMERGENCY,78603,Symptoms,2,False,207.0,NaN,NaN,NaN,NaN
1311,M,26,UNKNOWN/NOT SPECIFIED,EMERGENCY,7840,Symptoms,4,False,59.0,NaN,NaN,NaN,NaN


In [87]:
df_chapter_16 = pd.DataFrame(tables["780-799"])
df_chapter_16 = replace_nan_with_mean_or_preset(df_chapter_16)
# df_chapter_16.to_csv('chapter_16_nanx.csv', index=False)

In [88]:
def classify_icd9_XVII(code):

        # Extract the first three digits as an integer
        code_int = int(code[:3])  # Only first three digits

        if 800<= code_int <= 804:
            return "Fracture of skull"
        elif 805 <= code_int <= 809:
            return "Fracture of neck and trunk"
        elif 810 <= code_int <= 819:
            return "Fracture of upper limb"
        elif 820 <= code_int <= 829:
            return "Fracture of lower limb "
        elif 830 <= code_int <= 839:
            return "Dislocation"
        elif 840 <= code_int <= 848:
            return "Sprains and strains of joints and adjacent muscles"
        elif 850 <= code_int <= 854:
            return "Intracranial injury, excluding those with skull fracture"
        elif 860 <= code_int <= 869:
            return "Internal injury of thorax, abdomen, and pelvis"
        elif 870 <= code_int <= 879:
            return "Open wound of head, neck, and trunk"
        elif 880 <= code_int <= 889:
            return "Open wound of upper limb"
        elif 890 <= code_int <= 897:
            return "Open wound of lower limb"
        elif 900 <= code_int <= 907:
            return "Injury to blood vessels"
        elif 900 <= code_int <= 907:
            return "Late effects of injuries, poisonings, toxic effects, and other external causesInjury to blood vessels"
        elif 910 <= code_int <= 919:
            return "Superficial injury"
        elif 920 <= code_int <= 924:
            return "Contusion with intact skin surface "
        elif 925 <= code_int <= 929:
            return "Crushing injury "
        elif 930 <= code_int <= 939:
            return "Effects of foreign body entering through Body orifice  "
        elif 940 <= code_int <= 949:
            return "Burns (940–949) "
        elif 958 <= code_int <= 959:
            return "Certain traumatic complications and unspecified injuries"
        elif 960 <= code_int <= 979:
            return "Poisoning by drugs, medicinal and biological substances"
        elif 980 <= code_int <= 989:
            return "Toxic effects of substances chiefly nonmedicinal as to source"
        elif 990 <= code_int <= 995:
            return "Other and unspecified effects of external causes"
        elif 996 <= code_int <= 999:
            return "Complications of surgical and medical care, not elsewhere classified"
        else:
          return "Unknown"

tables["800-999"]['primary_icd9_label'] = tables["800-999"]['primary_icd9_code'].apply(classify_icd9_XVII)
tables["800-999"].head()

,gender,age,ethnicity,admission_type,primary_icd9_code,primary_icd9_label,num_of_icd9_codes,hospital_expire_flag,hospital_stay_hours,BMI,Hemoglobine,Hematocrit,WBC
0,M,43,WHITE,EMERGENCY,8601,"Internal injury of thorax, abdomen, and pelvis",4,False,172.0,NaN,NaN,NaN,NaN
5,M,29,BLACK/AFRICAN AMERICAN,EMERGENCY,86113,"Internal injury of thorax, abdomen, and pelvis",4,False,131.0,NaN,NaN,NaN,NaN
7,F,33,UNKNOWN/NOT SPECIFIED,EMERGENCY,85206,"Intracranial injury, excluding those with skul...",7,False,94.0,NaN,NaN,NaN,NaN
8,M,20,UNKNOWN/NOT SPECIFIED,EMERGENCY,80508,Fracture of neck and trunk,4,False,19.0,NaN,NaN,NaN,NaN
9,F,78,UNKNOWN/NOT SPECIFIED,EMERGENCY,99812,"Complications of surgical and medical care, no...",7,False,50.0,NaN,NaN,NaN,NaN


In [89]:
df_chapter_17 = pd.DataFrame(tables["800-999"])
df_chapter_17 = replace_nan_with_mean_or_preset(df_chapter_17)
# df_chapter_17.to_csv('chapter_17_nanx.csv', index=False)